<a href="https://colab.research.google.com/github/vvshyer/tensorflow2.0_learning/blob/master/cifar10_model_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import sklearn
import sys
import tensorflow as tf
import time

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

1.14.0
sys.version_info(major=3, minor=6, micro=8, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.4
pandas 0.24.2
sklearn 0.21.2
tensorflow 1.14.0
tensorflow.python.keras.api._v1.keras 2.2.4-tf


In [0]:
!pip install -q kaggle

In [0]:
from google.colab import files
files.upload()

In [0]:
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [0]:
!chmod 600 ~/.kaggle/kaggle.json
!kaggle datasets list

ref                                                         title                                                size  lastUpdated          downloadCount  
----------------------------------------------------------  --------------------------------------------------  -----  -------------------  -------------  
himanshupoddar/zomato-bangalore-restaurants                 Zomato Bangalore Restaurants                         88MB  2019-03-31 09:48:43           6335  
thegurus/spanish-high-speed-rail-system-ticket-pricing      Spanish High Speed Rail tickets pricing - Renfe      27MB  2019-05-10 06:50:30           1692  
taniaj/australian-election-2019-tweets                      Australian Election 2019 Tweets                      29MB  2019-05-21 09:41:38           1040  
brittabettendorf/berlin-airbnb-data                         Berlin Airbnb Data                                   89MB  2019-03-05 09:23:00           1998  
snocco/missing-migrants-project                             Miss

In [0]:
!kaggle competitions download -c cifar-10

  0% 0.00/3.04M [00:00<?, ?B/s]
100% 3.04M/3.04M [00:00<00:00, 99.3MB/s]
 99% 601M/610M [00:14<00:00, 88.4MB/s]
100% 610M/610M [00:14<00:00, 43.5MB/s]
 95% 99.0M/105M [00:02<00:00, 25.5MB/s]
100% 105M/105M [00:02<00:00, 38.6MB/s] 
  0% 0.00/575k [00:00<?, ?B/s]
100% 575k/575k [00:00<00:00, 146MB/s]


In [0]:
!ls
!7z x train.7z
!7z x test.7z

kaggle.json  sampleSubmission.csv  train.7z
sample_data  test.7z		   trainLabels.csv

7-Zip [64] 16.02 : Copyright (c) 1999-2016 Igor Pavlov : 2016-05-21
p7zip Version 16.02 (locale=en_US.UTF-8,Utf16=on,HugeFiles=on,64 bits,2 CPUs Intel(R) Xeon(R) CPU @ 2.30GHz (306F0),ASM,AES-NI)

Scanning the drive for archives:
  0M Scan         1 file, 109723070 bytes (105 MiB)

Extracting archive: train.7z
--
Path = train.7z
Type = 7z
Physical Size = 109723070
Headers Size = 294768
Method = LZMA:26
Solid = +
Blocks = 1

  0%      2%      2% 709 - train/10636.png                            2% 1215 - train/11091.png                             2% 1565 - train/11406.png                             2% 1863 - train/11675.png                             6% 186

In [0]:
class_names = [
    'airplane',
    'automobile',
    'bird',
    'cat',
    'deer',
    'dog',
    'frog',
    'horse',
    'ship',
    'truck',
]

train_labels_file = './trainLabels.csv'
test_csv_file = './sampleSubmission.csv'
train_folder = './train/'
test_folder = './test/'

def parse_csv_file(filepath, folder):
    """Parses csv file into (filename(path), label) format"""
    results = []
    with open(filepath, 'r') as f:
        lines = f.readlines()[1:]
    for line in lines:
        image_id, label_str = line.strip('\n').split(',')
        image_full_path = os.path.join(folder, image_id + '.jpg')
        results.append((image_full_path, label_str))
    return results

train_labels_info = parse_csv_file(train_labels_file, train_folder)
test_csv_info = parse_csv_file(test_csv_file, test_folder)

import pprint
pprint.pprint(train_labels_info[0:5])
pprint.pprint(test_csv_info[0:5])
print(len(train_labels_info), len(test_csv_info))

[('./train/1.jpg', 'frog'),
 ('./train/2.jpg', 'truck'),
 ('./train/3.jpg', 'truck'),
 ('./train/4.jpg', 'deer'),
 ('./train/5.jpg', 'automobile')]
[('./test/1.jpg', 'cat'),
 ('./test/2.jpg', 'cat'),
 ('./test/3.jpg', 'cat'),
 ('./test/4.jpg', 'cat'),
 ('./test/5.jpg', 'cat')]
50000 300000


In [0]:
# train_df = pd.DataFrame(train_labels_info)
train_df = pd.DataFrame(train_labels_info[0:45000])
valid_df = pd.DataFrame(train_labels_info[45000:])
test_df = pd.DataFrame(test_csv_info)

train_df.columns = ['filepath', 'class']
valid_df.columns = ['filepath', 'class']
test_df.columns = ['filepath', 'class']

print(train_df.head())
print(valid_df.head())
print(test_df.head())

        filepath       class
0  ./train/1.jpg        frog
1  ./train/2.jpg       truck
2  ./train/3.jpg       truck
3  ./train/4.jpg        deer
4  ./train/5.jpg  automobile
            filepath       class
0  ./train/45001.jpg       horse
1  ./train/45002.jpg  automobile
2  ./train/45003.jpg        deer
3  ./train/45004.jpg  automobile
4  ./train/45005.jpg    airplane
       filepath class
0  ./test/1.jpg   cat
1  ./test/2.jpg   cat
2  ./test/3.jpg   cat
3  ./test/4.jpg   cat
4  ./test/5.jpg   cat


In [0]:
!pwd

/content


In [0]:
height = 32
width = 32
channels = 3
batch_size = 32
num_classes = 10

train_datagen = keras.preprocessing.image.ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 40,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True,
    fill_mode = 'nearest',
)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = True,
    class_mode = 'sparse',
)

valid_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1./255)
valid_generator = valid_datagen.flow_from_dataframe(
    valid_df,
    directory = './',
    x_col = 'filepath',
    y_col = 'class',
    classes = class_names,
    target_size = (height, width),
    batch_size = batch_size,
    seed = 7,
    shuffle = False,
    class_mode = 'sparse',)

train_num = train_generator.samples
valid_num = valid_generator.samples
print(train_num, valid_num)

Found 0 validated image filenames belonging to 10 classes.
Found 0 validated image filenames belonging to 10 classes.
0 0


/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 45000 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  .format(n_invalid, x_col)
/usr/local/lib/python3.6/dist-packages/keras_preprocessing/image/dataframe_iterator.py:273: UserWarning: Found 5000 invalid image filename(s) in x_col="filepath". These filename(s) will be ignored.
  .format(n_invalid, x_col)


In [0]:
for i in range(2):
    x, y = train_generator.next()
    print(x.shape, y.shape)
    print(y)

(0, 32, 32, 3) (0,)
[]
(0, 32, 32, 3) (0,)
[]


In [0]:
model = keras.models.Sequential([
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same', 
                        activation='relu', input_shape=[width, height, channels]),
    keras.layers.Conv2D(filters=32, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same', 
                        activation='relu'),
    keras.layers.Conv2D(filters=64, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same', 
                        activation='relu'),
    keras.layers.Conv2D(filters=128, kernel_size=3, padding='same',
                        activation='relu'),
    keras.layers.MaxPool2D(pool_size=2),
    keras.layers.Flatten(),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(num_classes, activation='softmax'),
])

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam', metrics = ['accuracy'])

model.summary()

In [0]:
epochs = 300
history = model.fit_generator(train_generator,
                              steps_per_epoch = train_num // batch_size,
                              epochs = epochs,
                              validation_data = valid_generator,
                              validation_steps = valid_num//batch_size)

In [0]:
def plot_learning_curves(history, label, epochs, min_value, max_value):
    data = {}
    data[label] = history.history[label]
    data['val_'+label] = history.history['val_'+label]
    pd.DataFrame(data).plot(figsize=(8,5))
    plt.grid(True)
    plt.axis([0, epochs, min_value, max_value])
    plt.show()
    
plot_learning_curves(history, 'acc', epochs, 0, 1)
plot_learning_curves(history, 'loss', epochs, 1.5, 2.5)